In [37]:
import ghmm
import pickle
import numpy as np
from itertools import product
from pprint import pprint
import pysam
import os
import pandas
from copy import deepcopy
import re
import editdistance
import sys
from multiprocessing import Pool
%run "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/alignment_lib.ipynb"

In [38]:
NMERS = 3
NSTATES = 4**NMERS

In [39]:
args = {
    "events" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/david_events.template.pickle",
    "out_basename" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/david_calling",
    "ref": "/home/ibis/gregor.sturm/nanopore/david_eccles_bc_ideas/mouse_ref.fa",
    "hmm_params": "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/david_hmm_params_{0}mer.pickle".format(NMERS),
    "ncores": 4
}

# args = {
#     "events" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/lambda_events.template.pickle",
#     "out_basename" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/lambda_calling",
#     "ref": "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/lambda_ref.fasta",
#     "hmm_params": "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/lambda_hmm_params_{0}mer.pickle".format(NMERS),
#     "ncores": 62
# }

In [40]:
HMM_PARAMS = pickle.load(open(args["hmm_params"], 'rb'))
HMM_PARAMS = sorted(zip(*[HMM_PARAMS["kmers"], HMM_PARAMS["means"], HMM_PARAMS["stdv"]]))

# Train Model 

In [41]:
def mk_transmat(nmers):
    """make a transition matrix assuming move=1"""
    all_kmers = [x[0] for x in HMM_PARAMS]
    n_components = len(all_kmers)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(all_kmers):
        for i, to_kmer in enumerate(all_kmers):
            p = 1/4. if from_kmer[-(NMERS-1):] == to_kmer[:(NMERS-1)] else 0.
            transmat[j, i] = p          
            
    return transmat.tolist()

In [42]:
def mk_transmat0(nmers):
    """make a transition matrix assuming move=0 or move=1"""
    all_kmers = [x[0] for x in HMM_PARAMS]
    n_components = len(all_kmers)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(all_kmers):
        for i, to_kmer in enumerate(all_kmers):
            p = 0
            if from_kmer[-(NMERS-1):] == to_kmer[:(NMERS-1)]:
                """move=1"""
                p = (9/10.) * (1/4.) 
            elif from_kmer == to_kmer:
                """move=0"""
                p = (1/10.) * 1
            transmat[j, i] = p          
            
    return transmat.tolist()

In [43]:
def mk_transmat2(nmers):
    """make a transition matrix assuming move=0 or move=1 or move=2"""
    all_kmers = [x[0] for x in HMM_PARAMS]
    n_components = len(all_kmers)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(all_kmers):
        for i, to_kmer in enumerate(all_kmers):
            p = 0
            if from_kmer[-(NMERS-2):] == to_kmer[:(NMERS-2)]:
                """move=2"""
                p = (20/50.) * (1/16.)
            elif from_kmer[-(NMERS-1):] == to_kmer[:(NMERS-1)]:
                """move=1"""
                p = (29/50.) * (1/4.) 
            elif from_kmer == to_kmer:
                """move=0"""
                p = (1/50.) * 1
            transmat[j, i] = p          
            
    return transmat.tolist()

In [44]:
F = ghmm.Float()  # emission domain of this model

def mk_model():
    # example code for a continuous HMM with gaussian emissions



    A = mk_transmat2(NMERS)
    B = [[x[1], x[2]] for x in HMM_PARAMS]   # parameters of emission distributions in pairs of (mu, sigma)
    pi = [1/float(NSTATES)] * NSTATES   # initial probabilities per state

    # generate model from parameters
    model = ghmm.HMMFromMatrices(F,ghmm.GaussianDistribution(F), A, B, pi)
    return model

In [45]:
model = mk_model()
s = str(model)
print(s)

GaussianEmissionHMM(N=64)
  state 0 (initial=0.02, mu=70.17, sigma=3.84)
    Transitions: ->0 (0.03), ->1 (0.03), ->2 (0.03), ->3 (0.03), ->4 (0.03), ->5 (0.03), ->6 (0.03), ->7 (0.03), ->8 (0.03), ->9 (0.03), ->10 (0.03), ->11 (0.03), ->12 (0.03), ->13 (0.03), ->14 (0.03), ->15 (0.03)
  state 1 (initial=0.02, mu=63.14, sigma=3.47)
    Transitions: ->1 (0.02), ->4 (0.14), ->5 (0.14), ->6 (0.14), ->7 (0.14), ->16 (0.03), ->17 (0.03), ->18 (0.03), ->19 (0.03), ->20 (0.03), ->21 (0.03), ->22 (0.03), ->23 (0.03), ->24 (0.03), ->25 (0.03), ->26 (0.03), ->27 (0.03), ->28 (0.03), ->29 (0.03), ->30 (0.03), ->31 (0.03)

  ...

  state 62 (initial=0.02, mu=53.52, sigma=2.85)
    Transitions: ->32 (0.03), ->33 (0.03), ->34 (0.03), ->35 (0.03), ->36 (0.03), ->37 (0.03), ->38 (0.03), ->39 (0.03), ->40 (0.03), ->41 (0.03), ->42 (0.03), ->43 (0.03), ->44 (0.03), ->45 (0.03), ->46 (0.03), ->47 (0.03), ->56 (0.14), ->57 (0.14), ->58 (0.14), ->59 (0.14), ->62 (0.02)
  state 63 (initial=0.02, mu=49.86, s

In [46]:
def result_to_seq(result):
    states = result[0]
    all_kmers = [x[0] for x in HMM_PARAMS]
    kmers = [all_kmers[x] for x in states]
    seq = [kmer[NMERS/2] for kmer in kmers]
    return "".join(seq)

In [47]:
def predict(means):
    seq = ghmm.EmissionSequence(F, means)
    result = model.viterbi(seq)
    return result_to_seq(result)

In [48]:
predict([44.2, 44.3, 56, 58.2, 56.2, 58.1, 58.2, 60, 30.2])

u'TTTGTGTGT'

# Validate Model 

In [49]:
!pwd

/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline


In [50]:
assert os.path.isfile(args["events"])

In [51]:
ref_file = args["ref"]
test = !cat {ref_file} | grep ">"
print(test)
ref = !cat {ref_file} | grep -v ">"
ref = ref[0]
print(ref[:100])

['>mmusMT_PCR1']
GTTAATGTAGCTTAATAACAAAGCAAAGCACTGAAAATGCTTAGATGGATAATTGTATCCCATAAACACAAAGGTTTGGTCCTGGCCTTATAATTAATTA


In [52]:
file_data = pickle.load(open(args["events"], 'rb'))
file_data = [f for f in file_data if f is not None]

In [53]:
prepare_filemap(file_data)

### Make Alignment

In [54]:
fastq_file = "{0}.fastq".format(args["out_basename"])
mk_fastq(fastq_file, file_data)

In [55]:
sam_file = "{0}.sam".format(args["out_basename"])
graphmap(ref_file, fastq_file, sam_file, args["ncores"])

[Index 12:59:56] Running in fast and sensitive mode. Two indexes will be used (double memory consumption).
[Index 12:59:56] Index already exists. Loading from file.
[Index 12:59:58] Secondary index already exists. Loading from file.
[Index 12:59:59] Index loaded in 0.40 sec.
[Index 12:59:59] Memory consumption: [currentRSS = 513 MB, peakRSS = 513 MB]

[Run 12:59:59] Automatically setting the maximum allowed number of regions: max. 500, attempt to reduce after 100
[Run 12:59:59] Reference genome is assumed to be linear.
[Run 12:59:59] Only one alignment will be reported per mapped read.
[ProcessReads 12:59:59] Reads will be loaded in batches of up to 200 MB in size.
[ProcessReads 12:59:59] Batch of 81 reads (0 MiB) loaded in 0.01 sec. (19948664 bases)
[ProcessReads 12:59:59] Memory consumption: [currentRSS = 514 MB, peakRSS = 514 MB]
[ProcessReads 12:59:59] Using 4 threads.
[ProcessReads 13:00:00] [CPU time: 3.46 sec, RSS: 536 MB] Read: 81/81 (100.00%) [m: 80, u: 1]                     

In [56]:
bam_file = prepare_sam(args["out_basename"])

[samopen] SAM header is present: 1 sequences.


In [57]:
samfile = pysam.AlignmentFile(bam_file)
reads  = [x for x in samfile.fetch()]
len(reads)

80

In [58]:
def train_read(read_name): 
    file_id, channel_id = get_file_and_channel(read_name)
    file_obj = get_file(channel_id, file_id)      
    events = [x["mean"] for x in file_obj["events"].to_dict("records")]
    seq = ghmm.EmissionSequence(F, events)
    model.baumWelch(seq)
    

In [59]:
def basecall_read(params):
    read_name, ref_pos = params
    file_id, channel_id = get_file_and_channel(read_name)
#     sys.stdout.write("{0} {1} {2}\n".format(file_id, channel_id, read_name))
#     sys.stdout.flush()
    
    file_obj = get_file(channel_id, file_id)
      
    events = [x["mean"] for x in file_obj["events"].to_dict("records")]
    metrichor_seq = file_obj["fastq"].split("\n")[1]
    ref_seq = "".join([ref[x] for x in ref_pos])
    seq = ghmm.EmissionSequence(F, events)
    called_seq = predict(events)
    
    stats = {
        "d_metrichor": int(editdistance.eval(ref_seq, metrichor_seq)),
        "d_caller": int(editdistance.eval(ref_seq, called_seq)), 
        "length": len(ref_seq),
    }

    return (read_name, called_seq, stats)

In [ ]:
## one can not access a read in parallel (deadlock for whatever reason)
## therfore prepare input parameters outside of map
input_params = [(read.query_name, read.get_reference_positions()) for read in reads]
read_names = [x[0] for x in input_params]

In [ ]:
for i, read in enumerate(read_names): 
    sys.stdout.write('\rdone {0:%}'.format(i/float(len(read_names))))
    train_read(read)

done 3.750000%

In [ ]:
p = Pool(args["ncores"])

In [ ]:
print("Prediction: ")
results = []
try:
    for i, res in enumerate(p.imap_unordered(basecall_read, input_params), 1):
        results.append(res)
        sys.stdout.write('\rdone {0:%}'.format(i/float(len(input_params))))
    p.close()
    p.join()
except KeyboardInterrupt:
    p.terminate()

### Stats

In [ ]:
headers, seqs, stats = zip(*results)

In [ ]:
stats = pandas.DataFrame(list(stats))

In [ ]:
stats = stats.sum(0)

In [ ]:
stats

In [ ]:
print("Relative Performance: {0:5.3f}%".format(stats["d_metrichor"] * 100 /float(stats["d_caller"])))

In [ ]:
# ## Random control
# import random
# seqs = ["".join([random.choice("ACGT") for _ in range(len(read))]) for read in seqs]

In [ ]:
fasta_file_called = "{0}.called.fa".format(args["out_basename"])
with open(fasta_file_called, 'w') as f: 
    for header, seq in zip(headers, seqs): 
        f.write(">" + header + "\n")
        f.write(seq + "\n")

In [ ]:
sam_file_called = "{0}.called.sam".format(args["out_basename"])
graphmap(ref_file, fasta_file_called, sam_file_called, args["ncores"])

In [ ]:
prepare_sam("{0}.called".format(args["out_basename"]))